## 臉書載入頁面改版(2023.08.02)

### 遇到一次頁面改版，此情況下需要調整程式

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import re
import time

from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta

from urllib.parse import unquote
import urllib

## Load File

### 從Meta後台下載資料，並整理成符合格式的型態

In [95]:
file_name = input("Please enter file name(data date): ")

Please enter file name(data date): 20240215


In [96]:
df_meta = pd.read_csv(f'FB_{file_name}.csv', encoding = 'utf-8')

In [97]:
df_meta.head(3)

Post ID                                           Title  \
0  799883062173937                                             NaN   
1  799301062232137  退休老師45歲才開始存股，現在年領200萬股息！3個選股步驟、4個富習慣無私公開 - 今周刊   
2  799301518898758   退休後，千萬別被錢綁架！可幫子女「小錢」、別隨意給「大錢」，劃清界線才能開心花 - 今周刊   

                                         Description  \
0    超商「臨櫃」買咖啡的人很虧！\n7-11「咖啡好康」很多人不知道\n內行人曝買法：1年省破6千   
1  存股19年的高師大退休教授謝士英，45歲才開始學存股，他從220萬元開始投入，如今持股市值正...   
2  我不會幫子女出大錢，但平常卻很樂意出點小錢，譬如全家人出去聚餐，或只是和其中一個子女吃飯，都...   

                                              文章標題      Publish time  \
0  超商「臨櫃」買咖啡的人很虧！\n7-11「咖啡好康」很多人不知道\n內行人曝買法：1年省破6千  02/14/2024 15:02   
1   退休老師45歲才開始存股，現在年領200萬股息！3個選股步驟、4個富習慣無私公開 - 今周刊  02/14/2024 15:02   
2    退休後，千萬別被錢綁架！可幫子女「小錢」、別隨意給「大錢」，劃清界線才能開心花 - 今周刊  02/14/2024 16:02   

                                           Permalink Post type  Impressions  \
0  https://www.facebook.com/BToday/posts/pfbid0wr...      Text       166106   
1  https://www.facebook.com/BToday/posts/pfbid02y...      Link         9834   
2  https://www.facebook.com/BToday/posts/pfbid02Y...      Link        23849   

    Reach  Reactions  Shares  Comments  Total clicks  Other Clicks  \
0  166074        299      22        39         23365       23360.0   
1    9834         32      29         0           683          80.0   
2   23820        236      30        88          1509         802.0   

   Link Clicks  Matched Audience Targeting Consumption (Photo Click)  \
0          5.0                                                NaN      
1        603.0                                                NaN      
2        707.0                                                NaN      

   Negative feedback from users: Hide  Negative feedback from users: Hide all  
0                                 NaN                                     1.0  
1                                 NaN                                     NaN  
2                                 NaN                                     NaN

In [98]:
df_meta = df_meta.dropna(axis=0, how='all')
print(len(df_meta))

43


In [99]:
# choose data we need
lst_link = df_meta['Permalink']
lst_type = df_meta['Post type']

In [100]:
df_meta['Post type'].unique()

array(['Text', 'Link', 'Photo', 'Video'], dtype=object)

In [101]:
g_path = 'C:/Users/JoeJian/OneDrive - Business Today Magazine/桌面/Python Codes/Web_Scraping_in_Python_BeautifulSoup_Selenium&Scrapy_2023/ChromeDriver/chromedriver'


#### 從今周刊官網抓取所需要的資料

In [102]:
# Define get class function
# Beautiful Soup

## 不同縮網址，所帶出來的網頁元素不太一樣，故會有不同的抓取方式

def func_article_class(bt_link, bt_content_id, bt_class_main, bt_class_sub):
    try:
        time.sleep(2)
        
        print(bt_link)
        
        if "peview" in bt_link:
            print("wrong link: ", bt_link)
            bt_link = bt_link.replace("peview", "post")
            
        elif ('www.businesstoday.com.tw' in bt_link ) or ('bit.ly' in bt_link):
            bt_link = bt_link
        
        elif 'reurl.cc' in bt_link:
            headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
            other_response = requests.get(bt_link)

            other_url = BeautifulSoup(other_response.text, "lxml")
            bt_link = other_url.find("input", {"id":"url"}).get("value")
            print("reurl:", bt_link)
        
        else:
            headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
            other_response = requests.get(bt_link)

            other_url = BeautifulSoup(other_response.text, "lxml")

            all_a = other_url.find_all('a')

            for item in all_a:
                
                if 'businesstoday' in item.get('href'):
                    
                    bt_link = item.get('href')
                    print("Different url to original link:", bt_link)
        
        headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
        response = requests.get(bt_link)
        bt_html = BeautifulSoup(response.text, "lxml")
        
        # --- Begin 抓出文章id，之後會用到 ---
    
        content_id = bt_html.find("meta", {"property":"dable:item_id"}).get("content")
        bt_content_id.append(content_id)

        # --- End 抓出文章id，之後會用到 ---

        
        # --- Begin 文章分類 ---
        
        ct_class_main = bt_html.find("meta", {"property":"article:section"}).get("content")

        print(bt_html.find("meta", {"property":"article:section"}).get("content"))
        bt_class_main.append(ct_class_main)
        
        ct_class_sub = bt_html.find("meta", {"property":"article:section2"}).get("content")
        
        print(bt_html.find("meta", {"property":"article:section2"}).get("content"))
        bt_class_sub.append(ct_class_sub)
    
            
        # --- End 文章分類 ---
        
        
    except:
        pass
        print("nothing")
        
        bt_class_main.append(bt_link)
        bt_class_sub.append(bt_link)
        bt_content_id.append(bt_link)


#### Post type 為 Link時使用

In [103]:
# Define type Link scraping function

def func_link_class(fb_link, lst_class_main, lst_class_sub_ori, lst_content_id):
    """
    # first arg: Facebook link that you want to scrape
    # second arg: List that you append classfication, article attribution for Businesstoday
    # third arg: List that you append classfication, artcile class for Businesstoday
    
    """
    
    # -- Begin Get FB URL --

    web_fb = fb_link

    driver_fb = webdriver.Chrome('C:/Users/JoeJian/OneDrive - Business Today Magazine/桌面/Python Codes/Web_Scraping_in_Python_BeautifulSoup_Selenium&Scrapy_2023/ChromeDriver/chromedriver')
    
    
    driver_fb.get(web_fb)

    driver_fb.maximize_window()

    # -- End Get FB URL --
    
    # -- Begin Find to FB post Date & Time --
    
    ## 改版後，暫時抓不到發文時間
        
#     post_date_day_time(driver_fb, list_post_date, list_post_day, list_post_time)

#     list_post_date.append(" ")
#     list_post_day.append(" ")
#     list_post_time.append(" ")

    # -- End Find to FB post Date & Time --
    
    try:
        
        # -- Begin Find to BT Web Link in FB --
        
        time.sleep(0.5)

        web_bt_ori = driver_fb.find_element_by_xpath('//div[@class="xmjcpbm x1n2onr6"]/div/a').get_attribute('href')
        
        if '?u=http' in web_bt_ori:
            parsed_url = urllib.parse.urlparse(web_bt_ori)
            query_params = urllib.parse.parse_qs(parsed_url.query)
            web_bt = query_params["u"][0]
        else:
            web_bt = web_bt_ori

        print(web_bt)
        
        # -- End Find to BT Web Link in FB --

        # -- Quit FB Virtual Web --
        
        if 'thebetteraging' in (web_bt):

            link_class = '幸福熟齡'
            print(link_class)

            lst_class_main.append(link_class)
            lst_class_sub_ori.append(link_class)
            lst_content_id.append(" ")

        elif ('esg.businesstoday' in (web_bt)) or ('esgbusiness' in (web_bt)):

            link_class = 'ESG'
            print(link_class)

            lst_class_main.append(link_class)
            lst_class_sub_ori.append(link_class)
            lst_content_id.append(" ")

        else:
            
            func_article_class(web_bt, lst_content_id, lst_class_main, lst_class_sub_ori)
        
    except:
                
        print("except")
        
        lst_class_main.append(fb_link)
        lst_class_sub_ori.append(fb_link)
        lst_content_id.append(fb_link)
        
    # -- Quit FB Virtual Web --
    
    time.sleep(0.5)

    driver_fb.quit()

#### Post type 為text時使用

In [104]:
# Define type Text scraping function
    
# def func_text_class(fb_link, lst_class_main, lst_class_sub_ori, lst_content_id, list_post_date, list_post_day, list_post_time, post_type):
def func_text_class(fb_link, lst_class_main, lst_class_sub_ori, lst_content_id, post_type):
    """
    # first arg: Facebook link that you want to scrape
    # second arg: List that you append classfication, article attribution for Businesstoday
    # third arg: List that you append classfication, artcile class for Businesstoday
        
    """
    
    # -- Begin Get FB URL --

    web_fb = fb_link

    driver_fb = webdriver.Chrome('C:/Users/JoeJian/OneDrive - Business Today Magazine/桌面/Python Codes/Web_Scraping_in_Python_BeautifulSoup_Selenium&Scrapy_2023/ChromeDriver/chromedriver')

    driver_fb.get(web_fb)

    driver_fb.maximize_window()

    # -- End Get FB URL --
    
    # -- Begin Find to FB post Date & Time --
    
    ## 改版後，暫時抓不到發文時間

    # -- End Find to FB post Date & Time --
    
    try:
        try:
            time.sleep(0.5)

            ## 改版後，不用展開留言

            # -- Begin Get article link --
            time.sleep(0.5)

    #         web_bt_ori = driver_fb.find_element_by_xpath('//div[@class="x1lliihq xjkvuk6 x1iorvi4"]/span/div/div[2]/a').get_attribute('href')
            web_bt_ori = driver_fb.find_element_by_xpath('//div[@class="x4xrfw5 x9f619 x78zum5 xdt5ytf x1iyjqo2 xl56j7k x889kno xn6708d x1a8lsjc x1ye3gou xh8yej3"]/a').get_attribute('href')



        except:
            
            print("text version 1")
            
            ## 改版前，需要展開留言

            # Click comment button
            comment_button = WebDriverWait(driver_fb, 5).until(EC.presence_of_element_located((By.XPATH, "//span[@class='_1whp _4vn2']")))

            comment_button.click()

            # 2023/06/14 換成抓留言區的連結

            time.sleep(1)
            lst_fb_comment_all = driver_fb.find_elements_by_xpath("//div[@class='_72vr']")

#             lst_web_href = []

            for comment_num in range(len(lst_fb_comment_all)):

                print(comment_num)
                try:
                    print(lst_fb_comment_all[comment_num].find_element_by_xpath(".//a").text)

                    if lst_fb_comment_all[comment_num].find_element_by_xpath(".//a").text == "今周刊":

                        # 抓取今周刊留言的資料
                #                 bt_comment = lst_fb_comment_all[comment_num].text

                        lst_fb_href = lst_fb_comment_all[comment_num].find_elements_by_xpath(".//span/span/a")

                        for fb_href in lst_fb_href:
                            web_bt_ori = fb_href.get_attribute("href")
#                             lst_web_href.append(fb_href.get_attribute("href"))
                    break

                except:
                    pass

            print("text version 1 url:", web_bt_ori)
            
        parsed_url = urllib.parse.urlparse(web_bt_ori)
        query_params = urllib.parse.parse_qs(parsed_url.query)
        web_bt = query_params["u"][0]

        print(web_bt)
                
        # -- End Get article link --

        # -- Quit FB Virtual Web --
            
        if 'thebetteraging' in (web_bt):

            link_class = '幸福熟齡'
            print(link_class)
            
            lst_class_main.append(link_class)
            lst_class_sub_ori.append(link_class)
            lst_content_id.append(" ")

        elif ('esg.businesstoday' in (web_bt)) or ('esgbusiness' in (web_bt)):

            link_class = 'ESG'
            print(link_class)
            
            lst_class_main.append(link_class)
            lst_class_sub_ori.append(link_class)
            lst_content_id.append(" ")

        else:
            
            func_article_class(web_bt, lst_content_id, lst_class_main, lst_class_sub_ori)
        
    except:
        
        if ( post_type == 'Photo'):
            lst_class_main.append("圖片")
            lst_class_sub_ori.append("圖片")
            lst_content_id.append(" ")
        else:
            lst_class_main.append(fb_link)
            lst_class_sub_ori.append(fb_link)
            lst_content_id.append(fb_link)
        print("except")
        
    # -- Quit FB Virtual Web --
    
    time.sleep(0.5)

    driver_fb.quit()

In [105]:
# create list for article class
lst_article_class_main = []
lst_article_class_Ori = []
lst_article_id = []
# lst_post_date = []
# lst_post_day = []
# lst_post_time = []


### 開始爬取Facebook資料

In [106]:
link_num = 1

for link, post in zip(lst_link, lst_type):
    
    print("index:", link_num)
    link_num += 1
    
    if post == 'Link':

        func_link_class(link, lst_article_class_main, lst_article_class_Ori, lst_article_id)
#         func_link_class(link, lst_article_class_main, lst_article_class_Ori, lst_article_id, lst_post_date, lst_post_day, lst_post_time)
#         print(lst_post_date)

    elif ( post == 'Text' ) or ( post == 'Photo' ):
        
        func_text_class(link, lst_article_class_main, lst_article_class_Ori, lst_article_id, post)
#         func_text_class(link, lst_article_class_main, lst_article_class_Ori, lst_article_id, lst_post_date, lst_post_day, lst_post_time, post)
#         print(lst_post_date)
        
#     elif post == 'Photo':

#         lst_article_class_Ori.append('圖片')
    elif (post == 'Video') or (post == 'Live video'):
        
        print("影音")
        
#         # -- Begin Get FB URL --

#         driver_fb = webdriver.Chrome(ChromeDriverManager().install())

#         driver_fb.get(link)

#         driver_fb.maximize_window()       

#         # -- Begin Find to FB post Date & Time --

#         post_date_day_time(driver_fb, lst_post_date, lst_post_day, lst_post_time)
        
#         # -- End Find to FB post Date & Time --
        
        lst_article_class_main.append('影音')
        lst_article_class_Ori.append('影音')
        lst_article_id.append(" ")
        
#         driver_fb.quit()
        
#         # -- End Get FB URL --
#         lst_post_date.append(" ")
#         lst_post_day.append(" ")
#         lst_post_time.append(" ")
        
    else:
        print('other')
        
        lst_article_class_main.append('Other')
        lst_article_class_Ori.append('Other')
        lst_article_id.append(" ")
        
#         driver_fb.quit()
        
#         # -- End Get FB URL --
#         lst_post_date.append(" ")
#         lst_post_day.append(" ")
#         lst_post_time.append(" ")

print("------------ Finish -------------")

index: 1
https://www.businesstoday.com.tw/article/category/183030/post/202301080008/?utm_source=FB&utm_medium=article&utm_campaign=240215
https://www.businesstoday.com.tw/article/category/183030/post/202301080008/?utm_source=FB&utm_medium=article&utm_campaign=240215
職場生活
生活消費
index: 2
https://www.businesstoday.com.tw/article/category/183008/post/201912110005/?utm_source=FB&utm_medium=article&utm_campaign=240207
https://www.businesstoday.com.tw/article/category/183008/post/201912110005/?utm_source=FB&utm_medium=article&utm_campaign=240207
投資理財
台股
index: 3
https://www.businesstoday.com.tw/article/category/80401/post/201904100064/?utm_source=FB&utm_medium=article&utm_campaign=240207
https://www.businesstoday.com.tw/article/category/80401/post/201904100064/?utm_source=FB&utm_medium=article&utm_campaign=240207
投資理財
聰明理財
index: 4
https://reurl.cc/N4Qqz6
https://reurl.cc/N4Qqz6
reurl: https://www.businesstoday.com.tw/article/category/183012/post/202205130041/
投資理財
理財
index: 5
https://www.busi

https://www.businesstoday.com.tw/article/category/183027/post/202402150013/?utm_source=FB&utm_medium=article&utm_campaign=240215
產業時事
政治社會
index: 28
https://bit.ly/48oiVdT
https://bit.ly/48oiVdT
職場生活
生活消費
index: 29
https://www.businesstoday.com.tw/article/category/183027/post/202402150048/?utm_source=FB&utm_medium=article&utm_campaign=240215
https://www.businesstoday.com.tw/article/category/183027/post/202402150048/?utm_source=FB&utm_medium=article&utm_campaign=240215
產業時事
政治社會
index: 30
https://www.businesstoday.com.tw/article/category/183021/post/202402150047/?utm_source=FB&utm_medium=article&utm_campaign=20240215%E8%BE%B2%E6%9B%86%E5%B9%B4%E7%AF%80%E6%9C%9F%E9%96%93%EF%BC%8C%E6%9C%89200%E4%BD%8D%E5%8F%B0%E7%81%A3%E6%97%85%E5%AE%A2%E5%9C%A8%E8%B6%8A%E5%8D%97%E8%A2%AB%E4%B8%9F%E5%8C%85%EF%BC%8C%E8%A8%B1%E5%A4%9A%E6%B0%91%E7%9C%BE%E9%83%BD%E7%9B%B8%E7%95%B6%E5%A5%BD%E5%A5%87%EF%BC%8C%E9%81%87%E5%88%B0%E9%80%99%E7%A8%AE%E6%83%85%E6%B3%81%EF%BC%8C%E6%97%85%E9%81%8A%E4%B8%8D%E4%BE%BF%E9%9

In [107]:
lst_article_class_for_fb = []

In [108]:
# Define Business Today Classification
lst_產業動態 = ['產業動態', '科技', '傳產', '產業動態', '國際瞭望', '兩岸三地']
lst_金融保險 = ['金融保險', '金融', '保險', '稅制', '勞保勞退']
lst_國際總經 = ['國際總經']
lst_政治社會 = ['政治社會']
lst_投資理財 = ['投資理財', '台股', '全球股市', '債券基金', '信用卡', '理財', 'ETF', '聰明理財', '股債前線', '基金投資', '房地產掃瞄', '保險稅務']
lst_健康休閒 = ['健康休閒', '健康', '健康']
lst_生活消費 = ['生活消費', '美食旅遊', '品味收藏', '情感關係', '品味人生', '美食旅遊']
lst_職場學習 = ['職場學習', '職場', '教育', '退休', '個人成長', '管理', '行銷', '創業', '教育']
lst_家庭教育 = ['家庭教育']
lst_幸福熟齡 = ['幸福熟齡', '預約美好告別']
lst_房地產   = ['房地產']
lst_ESG      = ['ESG']
lst_訂閱制   = ['訂閱制']
lst_財經時事 = ['財經時事', '焦點新聞', '科技線上', '金融風雲']
lst_今選頻道 = ['今選頻道', '話題人物', '最新觀點', '名人專欄', '編輯室報告']

In [109]:
for item in lst_article_class_Ori:
    if item in lst_產業動態:
        lst_article_class_for_fb.append('產業動態')
        
    elif item in lst_金融保險:
        lst_article_class_for_fb.append('金融保險')
        
    elif item in lst_國際總經:
        lst_article_class_for_fb.append('國際總經')
        
    elif item in lst_政治社會:
        lst_article_class_for_fb.append('政治社會')
        
    elif item in lst_投資理財:
        lst_article_class_for_fb.append('投資理財')
        
    elif item in lst_健康休閒:
        lst_article_class_for_fb.append('健康休閒')

    elif item in lst_生活消費:
        lst_article_class_for_fb.append('生活消費')
        
    elif item in lst_職場學習:
        lst_article_class_for_fb.append('職場學習')
        
    elif item in lst_家庭教育:
        lst_article_class_for_fb.append('家庭教育')
        
    elif item in lst_幸福熟齡:
        lst_article_class_for_fb.append('幸福熟齡')
        
    elif item in lst_房地產:
        lst_article_class_for_fb.append('房地產')
        
    elif item in lst_ESG:
        lst_article_class_for_fb.append('ESG')
        
    elif item in lst_訂閱制:
        lst_article_class_for_fb.append('訂閱制')
        
    elif item in lst_今選頻道:
        lst_article_class_for_fb.append('今選頻道')
        
    else:
        lst_article_class_for_fb.append(item)

In [111]:
# print(lst_link)
print(len(lst_link))
print(len(lst_article_class_main))
# print(lst_article_class_Ori)
print(len(lst_article_class_Ori))
print(len(lst_article_class_for_fb))
print(len(lst_article_id))

# print(len(lst_post_date))
# print(len(lst_post_day))
# print(len(lst_post_time))

43
43
43
43
43


In [112]:
print(lst_article_class_main)

['職場生活', '投資理財', '投資理財', '投資理財', '投資理財', '投資理財', '投資理財', '產業時事', '產業時事', '投資理財', '產業時事', '產業時事', '投資理財', '投資理財', '產業時事', '投資理財', '職場生活', '職場生活', '職場生活', '產業時事', 'ESG', '產業時事', '職場生活', '幸福熟齡', '投資理財', '產業時事', '產業時事', '職場生活', '產業時事', '保險稅制', '幸福熟齡', '產業時事', '職場生活', '職場生活', '投資理財', '職場生活', '影音', '投資理財', '投資理財', '產業時事', '幸福熟齡', '投資理財', '職場生活']


In [113]:
print(lst_article_class_Ori)

['生活消費', '台股', '聰明理財', '理財', '台股', '全球股市', '台股', '金融', '金融', '理財', '金融', '政治社會', '台股', '台股', '政治社會', '台股', '退休', '健康', '生活消費', '政治社會', 'ESG', '國際總經', '生活消費', '幸福熟齡', '理財', '政治社會', '政治社會', '生活消費', '政治社會', '保險', '幸福熟齡', '政治社會', '健康', '生活消費', 'ETF', '美食旅遊', '影音', '台股', '房地產', '科技', '幸福熟齡', '房地產', '情感關係']


In [114]:
print(lst_article_class_for_fb)

['生活消費', '投資理財', '投資理財', '投資理財', '投資理財', '投資理財', '投資理財', '金融保險', '金融保險', '投資理財', '金融保險', '政治社會', '投資理財', '投資理財', '政治社會', '投資理財', '職場學習', '健康休閒', '生活消費', '政治社會', 'ESG', '國際總經', '生活消費', '幸福熟齡', '投資理財', '政治社會', '政治社會', '生活消費', '政治社會', '金融保險', '幸福熟齡', '政治社會', '健康休閒', '生活消費', '投資理財', '生活消費', '影音', '投資理財', '房地產', '產業動態', '幸福熟齡', '房地產', '生活消費']


In [115]:
print(lst_article_id)

['202301080008', '201912110005', '201904100064', '202205130041', '202402150003', '202402150001', '202402050018', '202312130050', '202312130048', '202402150004', '202402020023', '202402150007', '202402150012', '202402070003', '202402150013', '202402150014', '202402020038', '202102170010', '202402150029', '202402150019', ' ', '202402150024', '202402150028', ' ', '202402150010', '202402150044', '202402150013', '202402150030', '202402150048', '202402150047', ' ', '202402150031', '202402060040', '202402150046', '202402150008', '202402150002', ' ', '202402150037', '202402150009', '202402150045', '201903220027', '202402150027', '202205070007']


In [116]:
## 如果資料斷掉時使用，就不用全部從來

# len(lst_link[:25])
# lst_link = lst_link[:25]
# print(len(lst_link))

In [ ]:
# print(lst_post_date)
# print(lst_post_day)
# print(lst_post_time)

### 從後台下載臉書資料，取得發布時間

In [ ]:
df_fb_report = pd.read_csv('FbRepot.csv', encoding = 'utf-8')

df_fb_report.head(5)

In [119]:
lst_post_date = []
lst_post_day = []
lst_post_time = []

In [120]:
for post_time in df_fb_report['發布時間']:
    
    date_str, time_str = post_time.split(' ')[0], post_time.split(' ')[1]
#     print('date_str: ', date_str)
#     print('time_str: ', time_str)

    post_YYYY = int(date_str.split('-')[0])
    post_MM = int(date_str.split('-')[1])
    post_DD = int(date_str.split('-')[2])

    day_str = datetime(post_YYYY, post_MM, post_DD)
    day_str = day_str.strftime("%w")
    if day_str == "0":
        day_str = "7"
    
    date_str = date_str.replace("-", "/")
    
    print(date_str, " ", day_str, " ", time_str)

    lst_post_date.append(date_str)
    lst_post_day.append(day_str)
    lst_post_time.append(time_str)
    

2024/02/13   2   20:30:52
2024/02/12   1   18:21:55
2024/02/12   1   11:00:01
2024/02/11   7   18:00:00
2024/02/15   4   17:44:25
2024/02/15   4   07:30:01
2024/02/12   1   17:09:49
2024/02/12   1   11:24:13
2024/02/11   7   21:30:00
2024/02/13   2   21:41:01
2024/02/11   7   19:30:01
2024/02/11   7   11:00:01
2024/02/12   1   10:10:00
2024/02/13   2   23:00:02
2024/02/13   2   05:00:00
2024/02/14   3   19:30:01
2024/02/14   3   11:30:01
2024/02/14   3   18:30:00
2024/02/15   4   16:10:15
2024/02/12   1   19:00:01
2024/02/14   3   11:00:01
2024/02/15   4   15:39:32
2024/02/11   7   09:00:01
2024/02/13   2   10:30:01
2024/02/12   1   09:00:01
2024/02/13   2   19:00:02
2024/02/14   3   13:00:01
2024/02/14   3   16:30:01
2024/02/16   5   07:50:01
2024/02/13   2   08:30:00
2024/02/11   7   16:00:00
2024/02/15   4   20:10:01
2024/02/14   3   22:00:01
2024/02/11   7   12:30:00
2024/02/15   4   22:00:01
2024/02/11   7   13:00:01
2024/02/15   4   19:00:02
2024/02/15   4   19:15:20
2024/02/15  

In [121]:
df_post_time = pd.DataFrame({ 'Post ID': df_fb_report['FB post ID']
                            , '發文日期': lst_post_date
                            , '發文星期': lst_post_day
                            , '發文時間': lst_post_time
                            , 'Post_date_time': df_fb_report['發布時間']})

In [122]:
df_post_time.head(5)

Post ID        發文日期 發文星期      發文時間       Post_date_time
0  796011975894379  2024/02/13    2  20:30:52  2024-02-13 20:30:52
1  804996488329261  2024/02/12    1  18:21:55  2024-02-12 18:21:55
2  795966995898877  2024/02/12    1  11:00:01  2024-02-12 11:00:01
3  789298293232414  2024/02/11    7  18:00:00  2024-02-11 18:00:00
4  806934644802112  2024/02/15    4  17:44:25  2024-02-15 17:44:25

In [123]:
df_new = pd.DataFrame({ 'Permalink': lst_link
                       ,'文章ID': lst_article_id
                       ,'文章大分類': lst_article_class_main
                       ,'文章次分類': lst_article_class_Ori
                       ,'文章屬性_2': lst_article_class_for_fb})
#                        ,'發文日期': lst_post_date
#                        ,'發文星期': lst_post_day
#                        ,'發文時間': lst_post_time})

df_meta_new = pd.merge(df_meta, df_new)

df_meta_new = pd.merge(df_meta_new, df_post_time, how = 'left')
                       
df_meta_new

Post ID                                              Title  \
0   799883062173937                                                NaN   
1   799301062232137     退休老師45歲才開始存股，現在年領200萬股息！3個選股步驟、4個富習慣無私公開 - 今周刊   
2   799301518898758      退休後，千萬別被錢綁架！可幫子女「小錢」、別隨意給「大錢」，劃清界線才能開心花 - 今周刊   
3   792132879615622      連巴菲特請客麥當勞也會用折價券！財務自由10習慣：想致富，你得先換上有錢人的腦袋- 今周刊   
4   806700078158902                                                NaN   
5   806693098159600                                                NaN   
6   800625635433013                                                NaN   
7   801166225378954  南山人壽尹崇堯點名它能改變台灣，貝萊德與私募大咖KKR爭先搶進…「影響力投資」有何魔力？- 今周刊   
8   802346258594284  50年前賺錢方式，如今世界已經不歡迎了！國泰金控程淑芬看企業永續轉型：「No ESG, no...   
9   806736754821901                                                NaN   
10  806691664826410                                                NaN   
11  806731854822391                                                NaN   
12  806749431487300  龍年開紅盤／台積電(2330)攻破7字頭、台股飆漲629點史上第2大！歷史新高的台股還能買？...   
13  804836908345219  龍年股債兩頭賺！高股息ETF仍有五年好光景？ 謝富旭：買債分層次，4.5%到7%甜甜價這樣算...   
14  806756364819940                                                NaN   
15  806765911485652   今周重磅》MSCI剔除66家中企，資金出逃流入台股？台積電688學長解套，輝達下周財報能再助攻？   
16  806697084825868  退休後才匆忙進場買股票的人最危險！一下單就是40萬，當年證券行裡有錢老太太給我的投資啟示- 今周刊   
17  806762748152635                                                NaN   
18  806882874807289                                                NaN   
19  806843808144529  豬肉含西布特羅706件未檢出，僅台中唯1...禁藥過去2年輸入789毫克，保七警察介入查流向...   
20  801117512050492                                                NaN   
21  806885528140357                                                NaN   
22  806894961472747                                                NaN   
23  806877211474522  「窮人的婚姻就是一場合租」父親中風婆媳失和，離婚後，「我仍相信自己可以活成令人羨慕的樣子！」...   
24  806757544819822  股神巴菲特開始賣蘋果！波克夏Q4減持1%、這檔也砍7成...前十大持股一次看，最新投資仍保密...   
25  806934644802112                                                NaN   
26  806919774803599                                                NaN   
27  806902734805303                                                NaN   
28  806948724800704                                                NaN   
29  806943064801270      海外旅遊被丟包，旅遊不便險會賠？被逼刷卡算「爭議款項」能不繳？金管會給正確答案 - 今周刊   
30  806879518140958  勇敢求職吧，這世界比你想像更美好！中高齡重返職場，除了自己快樂、經濟獨立，也做回自己 - 幸福熟齡   
31  806922671469976                                                NaN   
32  806699421492301   年後減重》過年吃胖的小凸肚該怎麼減？資深營養師激推4食物：打開「燃脂開關」減少慢性發炎- 今周刊   
33  806942461467997                                                NaN   
34  806743541487889                                                NaN   
35  806724704823106     一個人獨行新店溪、碧潭，享受過年暖冬！登和美山頂俯瞰巿區，謝金河讚嘆：天際線更遼闊- 今周刊   
36  796013625894214                      【83歲阿伯為台灣寫歌半世紀 榮登世界頂尖100音樂家！】   
37  806923784803198                                                NaN   
38  806893861472857  情侶婚前合買房屋「3件必做的事」！出資比例、誰當借款人…一文讀懂不吃虧，不怕分手對簿公堂- 今周刊   
39  806947488134161  乳癌連19年居台灣女性癌症首位！台康生技耗十年做出首款生物相似藥　為何曾馨瑩說「盼乳癌能變成...   
40  799296855565891         謝哲青的歐洲攻略！當日常被煩悶掏空，你需要去旅行─此生必玩的3個美麗國家 - 今周刊   
41  806878061474437                                                NaN   
42  799295188899391  「如果你害怕終有一天要跟母親告別⋯」導演王希捷花7年寫一封「獻給全台媽媽的情書」：堅強的她，...   

                                          Description  \
0     超商「臨櫃」買咖啡的人很虧！\n7-11「咖啡好康」很多人不知道\n內行人曝買法：1年省破6千   
1   存股19年的高師大退休教授謝士英，45歲才開始學存股，他從220萬元開始投入，如今持股市值正...   
2   我不會幫子女出大錢，但平常卻很樂意出點小錢，譬如全家人出去聚餐，或只是和其中一個子女吃飯，都...   
3   🎯Podcast聽財務自由的好習慣👉https://reurl.cc/RWqvxZ\n-\n...   
4                      台股龍年開紅盤狂飆600點！台積電差一檔漲停奔709元新天價   
5            台積電ADR狂衝1.35％、輝達市值超車Google母公司「擠進全美前3強企業」   
6                          3檔增資股今天起開抽「只有3天」，申購全中能賺4萬！   
7   「能力愈大，責任愈大⋯⋯。」說話的，是南山人壽董事長尹崇堯，地點，在十二月七日的聯合國氣候變...   
8   兩年前開始參加COP的國泰金控投資長程淑芬分享，過去金融業是這場峰會的局外人。\n\n如今，...   
9                   30年新低！\n日圓「0.2123」甜甜價\n換10萬台幣多賺1萬   
10    玩期貨半年慘賠千萬，他連活下去的信心都跌光，幸虧遇到好老婆「你才欠1千多萬，有什麼了不起？！」   
11       富國島上百名旅客被迫刷2.2萬，可列爭議款？年代、越南旅行社各說各話，真該自己出面處理？   
12  台股龍年開紅盤，週一

In [124]:
df_meta_new.to_excel(f'FB_{file_name}_classification.xlsx', encoding = 'utf-8-sig', index = False)

C:\Users\JoeJian\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
